# Database Retrieve and Clean

### This file can be run with "run all"

### Table of Contents
1. [Legacy Store Details](#legacy-store-details)
1. [Legacy Users](#legacy-users)
1. [Orders Table](#orders-table)
1. [Card Details](#card-details)
1. [Store Details](#store-details)
1. [Products Data](#products-data)
1. [Date Events Data](#date-events-data)

In [1]:
# ALLWAYS RUN FIRST

from database_utils import DatabaseConnector
from data_extraction import DataExtraction
from data_cleaning import DataCleaning
import pandas as pd

database_connector = DatabaseConnector()
data_extractor = DataExtraction()
data_cleaner = DataCleaning()

In [ ]:
database_connector.list_db_tables()

## Legacy Store Details

In [ ]:
# Reads a database table as "legacy_store_details_df" and saves it as a csv
legacy_store_details_df = data_extractor.read_rds_table("legacy_store_details")
legacy_store_details_df.to_csv("original_dfs/original_legacy_store_details_df.csv")

In [ ]:
# Prints details of the "legacy_store_details_df"
legacy_store_details_df.info()
legacy_store_details_df.columns

In [ ]:
# Cleans the legacy_store_details_data
cleaned_legacy_store_details_df = legacy_store_details_df
cleaned_legacy_store_details_df = cleaned_legacy_store_details_df.drop("lat", axis=1)
cleaned_legacy_store_details_df = data_cleaner.clean_null(cleaned_legacy_store_details_df)
cleaned_legacy_store_details_df = cleaned_legacy_store_details_df[['address', 'longitude', 'latitude', 'locality', 'store_code', 'staff_numbers', 'opening_date', 'store_type', 'country_code', 'continent']]
cleaned_legacy_store_details_df = data_cleaner.clean_rows_by_length_condition(cleaned_legacy_store_details_df, "country_code", 3)
cleaned_legacy_store_details_df = data_cleaner.clean_convert_date_column(cleaned_legacy_store_details_df, "opening_date")
cleaned_legacy_store_details_df = data_cleaner.clean_replace_value_in_column(cleaned_legacy_store_details_df, "continent", "eeAmerica", "America")
cleaned_legacy_store_details_df = data_cleaner.clean_replace_value_in_column(cleaned_legacy_store_details_df, "continent", "eeEurope", "Europe")

cleaned_legacy_store_details_df.info()
cleaned_legacy_store_details_df.to_csv("cleaned_dfs/cleaned_legacy_store_details_df.csv")

## Legacy Users

In [ ]:
# Reads a database table as "legacy_users_df" and saves it as a csv
legacy_users_df = data_extractor.read_rds_table("legacy_users")
legacy_users_df.to_csv("original_dfs/original_legacy_users_df.csv")

# Prints details of the "legacy_users_df"
legacy_users_df.info()
legacy_users_df.columns

In [ ]:
# Starts a new cleaned variable
cleaned_legacy_users_df = legacy_users_df

# Cleans the legacy_users_data
cleaned_legacy_users_df = data_cleaner.clean_rows_by_length_condition(cleaned_legacy_users_df, "country_code", 3)
cleaned_legacy_users_df = data_cleaner.clean_null(cleaned_legacy_users_df)
cleaned_legacy_users_df = data_cleaner.clean_replace_value_in_column(cleaned_legacy_users_df, "country_code", "GGB", "GB")
cleaned_legacy_users_df = data_cleaner.clean_convert_date_column(cleaned_legacy_users_df, "date_of_birth")
cleaned_legacy_users_df = data_cleaner.clean_convert_date_column(cleaned_legacy_users_df, "join_date")
# cleaned_legacy_users_df = cleaned_legacy_users_df.reset_index(drop=True)    # Resets the index

cleaned_legacy_users_df.info()

# Overwrites the cleaned_df.csv witht the current cleared_df 
cleaned_legacy_users_df.to_csv("cleaned_dfs/cleaned_legacy_users_df.csv")

# Uploads to sales_data
database_connector.upload_to_db(cleaned_legacy_users_df, "dim_users")


## Orders Table

In [2]:
# Reads a database table as "orders_table_df" and saves it as a csv
orders_table_df = data_extractor.read_rds_table("orders_table")
orders_table_df.to_csv("original_dfs/original_orders_table_df.csv")

# Prints details of the "orders_table_df"
orders_table_df.info()
orders_table_df.columns

<class 'pandas.core.frame.DataFrame'>
Index: 120123 entries, 0 to 118804
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   level_0           120123 non-null  int64 
 1   date_uuid         120123 non-null  object
 2   first_name        15284 non-null   object
 3   last_name         15284 non-null   object
 4   user_uuid         120123 non-null  object
 5   card_number       120123 non-null  int64 
 6   store_code        120123 non-null  object
 7   product_code      120123 non-null  object
 8   1                 0 non-null       object
 9   product_quantity  120123 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 10.1+ MB


Index(['level_0', 'date_uuid', 'first_name', 'last_name', 'user_uuid',
       'card_number', 'store_code', 'product_code', '1', 'product_quantity'],
      dtype='object')

In [ ]:
# Starts a new cleaned variable
cleaned_orders_table_df = orders_table_df

# Cleans the orders_table_df
cleaned_orders_table_df = cleaned_orders_table_df.drop(["level_0", "1", "first_name", "last_name"], axis=1)

cleaned_orders_table_df.info()

# Overwrites the cleaned_df.csv witht the current cleared_df 
cleaned_orders_table_df.to_csv("cleaned_dfs/cleaned_orders_table_df.csv")

# Uploads to sales_data
database_connector.upload_to_db(cleaned_orders_table_df, "orders_table")


## Card Details

In [ ]:
# Retrieve data from pdf and save to pdf
card_details_df = data_extractor.retrieve_pdf_data()
card_details_df.to_csv("original_dfs/original_card_details.csv")

# Prints details of the "card_details_df"
card_details_df.info()
card_details_df.columns

In [ ]:
# Starts a new cleaned variable
cleaned_card_details_df = card_details_df

# Cleans the "card_details_df"
cleaned_card_details_df = data_cleaner.clean_null(cleaned_card_details_df)
cleaned_card_details_df = data_cleaner.clean_rows_by_length_condition(cleaned_card_details_df, "expiry_date", 5)
cleaned_card_details_df['expiry_date'] = pd.to_datetime(cleaned_card_details_df['expiry_date'], format='%m/%y')
cleaned_card_details_df = data_cleaner.clean_convert_date_column(cleaned_card_details_df, "date_payment_confirmed")

cleaned_card_details_df.info()

# Overwrites the cleaned_df.csv with the current cleared_df 
cleaned_card_details_df.to_csv("cleaned_dfs/cleaned_card_details_df.csv")

# Uploads to sales_data
database_connector.upload_to_db(cleaned_card_details_df, "dim_card_details")


## Store Details

In [ ]:
# Lists number of stores in the store data
data_extractor.list_number_of_stores()


In [7]:
## TAKES 10 MINMUTES!!!!
store_data_df = data_extractor.retrieve_stores_data()
store_data_df
store_data_df.to_csv("original_dfs/original_store_data.csv")

# Note: This table appears to be the same as "legacy_store_details_df" although I have treated is is if it was not.

In [11]:
# Starts a new cleaned variable
cleaned_store_data_df = store_data_df

# Cleans the store_data_df
cleaned_store_data_df = cleaned_store_data_df.drop("lat", axis=1)
cleaned_store_data_df = data_cleaner.clean_null(cleaned_store_data_df)
cleaned_store_data_df = cleaned_store_data_df[['address', 'longitude', 'latitude', 'locality', 'store_code', 'staff_numbers', 'opening_date', 'store_type', 'country_code', 'continent']]
cleaned_store_data_df = data_cleaner.clean_rows_by_length_condition(cleaned_store_data_df, "country_code", 3)
cleaned_store_data_df = data_cleaner.clean_convert_date_column(cleaned_store_data_df, "opening_date")
cleaned_store_data_df = data_cleaner.clean_replace_value_in_column(cleaned_store_data_df, "continent", "eeAmerica", "America")
cleaned_store_data_df = data_cleaner.clean_replace_value_in_column(cleaned_store_data_df, "continent", "eeEurope", "Europe")
cleaned_store_data_df = data_cleaner.clean_replace_value_in_column(cleaned_store_data_df, "staff_numbers", "30e", 30)
cleaned_store_data_df = data_cleaner.clean_replace_value_in_column(cleaned_store_data_df, "staff_numbers", "A97", 97)
cleaned_store_data_df = data_cleaner.clean_replace_value_in_column(cleaned_store_data_df, "staff_numbers", "80R", 80)
cleaned_store_data_df = data_cleaner.clean_replace_value_in_column(cleaned_store_data_df, "staff_numbers", "J78", 78)
cleaned_store_data_df = data_cleaner.clean_replace_value_in_column(cleaned_store_data_df, "staff_numbers", "3n9", 39)

cleaned_store_data_df.info()
cleaned_store_data_df.to_csv("cleaned_dfs/cleaned_store_data_df.csv")

# Uploads to sales_data
database_connector.upload_to_db(cleaned_store_data_df, "dim_store_details")


<class 'pandas.core.frame.DataFrame'>
Index: 440 entries, 1 to 450
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   address        440 non-null    object        
 1   longitude      440 non-null    object        
 2   latitude       440 non-null    object        
 3   locality       440 non-null    object        
 4   store_code     440 non-null    object        
 5   staff_numbers  440 non-null    object        
 6   opening_date   440 non-null    datetime64[ns]
 7   store_type     440 non-null    object        
 8   country_code   440 non-null    object        
 9   continent      440 non-null    object        
dtypes: datetime64[ns](1), object(9)
memory usage: 37.8+ KB


## Products Data

In [ ]:
# downloads and saves the products data
data_extractor.extract_from_s3()

# prints details about the products data
products_data_df = pd.read_csv("original_dfs/original_products.csv")
products_data_df.info()
products_data_df.columns

In [ ]:
# Starts a new cleaned variable
cleaned_products_data_df = products_data_df

# cleans data in products_data_df
cleaned_products_data_df = cleaned_products_data_df.drop("Unnamed: 0", axis=1)
cleaned_products_data_df = data_cleaner.clean_null(cleaned_products_data_df)
cleaned_products_data_df = data_cleaner.clean_rows_by_length_condition(cleaned_products_data_df, "product_price", 7)
cleaned_products_data_df = data_cleaner.clean_replace_value_in_column(cleaned_products_data_df, "weight", "77g .", "77g")
cleaned_products_data_df = data_cleaner.clean_convert_date_column(cleaned_products_data_df, "date_added")

# Apply the conversion function to the 'Weight' column
cleaned_products_data_df['weight_in_kg'] = cleaned_products_data_df['weight'].apply(data_cleaner.convert_product_weight)

cleaned_products_data_df.info()
cleaned_products_data_df.to_csv("cleaned_dfs/cleaned_products_data_df.csv")

# Uploads to sales_data
database_connector.upload_to_db(cleaned_products_data_df, "dim_products")


## Date Events Data

In [ ]:
# Reads JSON and writes to csv
date_events_df = pd.read_json("https://data-handling-public.s3.eu-west-1.amazonaws.com/date_details.json")
date_events_df.to_csv("original_dfs/original_date_events_df.csv")

# Print details on the "date_events_df"
date_events_df.info()
date_events_df.columns

In [ ]:
# Starts a new cleaned variable
cleaned_date_events_df = date_events_df

# cleans data in date_events_df
cleaned_date_events_df = data_cleaner.clean_null(cleaned_date_events_df)
cleaned_date_events_df = data_cleaner.clean_rows_by_length_condition(cleaned_date_events_df, "month", 3)
cleaned_date_events_df = data_cleaner.concatenate_columns(cleaned_date_events_df, "date", ["year", "month", "day"], "-")
cleaned_date_events_df = data_cleaner.concatenate_columns(cleaned_date_events_df, "datetime", ["date", "timestamp"], " ")
cleaned_date_events_df = data_cleaner.clean_convert_date_column(cleaned_date_events_df, "date")
cleaned_date_events_df = data_cleaner.clean_convert_date_column(cleaned_date_events_df, "datetime")

cleaned_date_events_df.info()
cleaned_date_events_df.to_csv("cleaned_dfs/cleaned_date_events_df.csv")

# Uploads to sales_data
database_connector.upload_to_db(cleaned_date_events_df, "dim_date_time")
